In [1]:
suppressWarnings(suppressMessages(library(tidyverse)))
suppressWarnings(suppressMessages(require(reshape)))
suppressWarnings(suppressMessages(library(fdrtool)))
 normalize <- function(x){
  return((x-mean(x,na.rm = TRUE)) / (sd(x,na.rm = TRUE)))
}

In [2]:
filter_genes <- read.table("../../reports/Master Table/Total_Counts_master_table_Median_long_v2.tab",header=TRUE)

In [3]:
filter_genes <- filter_genes%>%mutate(Effect=effect)

In [4]:
head(filter_genes,2)

,strain,condition,slevel,locus_tag,Median,controlMedian,effect,log2_coefficient,mean,controlmean,fdr,Annotation,Effect
,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
1,DCLPA,canavanine,HIGH,CCNA_00001,0,0.0,strainDCLPA:conditionCAN:slevelHIGH,-1.444299,0.3333333,1093.75,0.0730416,Essential,strainDCLPA:conditionCAN:slevelHIGH
2,DCLPA,canavanine,HIGH,CCNA_00002,7603,5948.5,strainDCLPA:conditionCAN:slevelHIGH,NA,7800.0000000,6192.25,NA,Conditionally_Neutral,strainDCLPA:conditionCAN:slevelHIGH


In [5]:
 f_genes <- filter_genes%>%filter(Annotation=="Conditionally_Essential"|Annotation== "Essential")%>%select(strain,condition,slevel,locus_tag)%>%mutate(nonmodel="Yes")

In [6]:
##

In [7]:
TC <- read.table("../../reports/HORSHOE_LFSR/GLMNET_TC_RESULTS.TAB",header=TRUE,check.names=FALSE,fill=TRUE)
TC <- melt(TC,id="locus_tag")
colnames(TC) <- c("locus_tag","Effect","Estimate")

In [8]:
head(TC,2)

,locus_tag,Effect,Estimate
,<chr>,<fct>,<dbl>
1,CCNA_00001,Intercept,5.434611
2,CCNA_00002,Intercept,8.471513


In [9]:
#%>%filter(condition!="NA" & slevel!="NA")

In [10]:
TC_filter <- TC%>%separate(Effect, c("strain","condition","slevel"), ":",remove=FALSE)%>%filter(condition!="NA" & slevel!="NA")%>%mutate(strain=gsub("strain","",strain),
                                                                                                                                   condition=gsub("condition","",condition),
                                                                                                                                   slevel=gsub("slevel","",slevel))%>%
filter(Effect!="Intercept")%>%mutate(condition=gsub("CONTROL","none",condition))

Warning message:
"Expected 3 pieces. Missing pieces filled with `NA` in 81680 rows [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, ...]."


In [11]:
head(TC_filter)

,locus_tag,Effect,strain,condition,slevel,Estimate
,<chr>,<fct>,<chr>,<chr>,<chr>,<dbl>
1,CCNA_00001,strainwild-type:conditionheat:slevelLOW,wild-type,heat,LOW,-0.12980586
2,CCNA_00002,strainwild-type:conditionheat:slevelLOW,wild-type,heat,LOW,0.03576769
3,CCNA_00003,strainwild-type:conditionheat:slevelLOW,wild-type,heat,LOW,-0.44379345
4,CCNA_00004,strainwild-type:conditionheat:slevelLOW,wild-type,heat,LOW,-0.22833234
5,CCNA_00005,strainwild-type:conditionheat:slevelLOW,wild-type,heat,LOW,-0.01458529
6,CCNA_00006,strainwild-type:conditionheat:slevelLOW,wild-type,heat,LOW,-0.09743234


In [12]:
unique(TC_filter$strain)

[1] "wild-type" "DLON"      "dnak-dnaJ" "DCLPA"     "DCLPB"

In [13]:
unique(TC_filter$condition)

[1] "heat"               "oxidative-peroxide" "canavanine"

In [14]:
unique(TC_filter$slevel)

[1] "LOW"    "MEDIUM" "HIGH"

In [15]:
unique(f_genes$strain)

[1] "DCLPA"     "DCLPB"     "DLON"      "dnak-dnaJ" "wild-type"

In [16]:
unique(f_genes$condition)

[1] "canavanine"         "heat"               "none"              
[4] "oxidative-peroxide"

In [17]:
unique(f_genes$slevel)

[1] "HIGH"   "LOW"    "MEDIUM" "none"

In [18]:
#TC_filter <- TC%>%separate(Effect, c("strain","condition","slevel"), ":",remove=FALSE)%>%drop_na()%>%filter(condition!="conditionnone")%>%mutate(strain=gsub("strain","",strain),
                                                                                                                                 #  condition=gsub("condition","",condition),
                                                                                                                                 #  slevel=gsub("slevel","",slevel))

In [19]:
nrow(f_genes)

[1] 41221

In [20]:
nrow(TC_filter)

[1] 183780

In [21]:
unique(f_genes$condition)

[1] "canavanine"         "heat"               "none"              
[4] "oxidative-peroxide"

In [22]:
TC <- merge(TC_filter,f_genes,by=c("strain","condition","slevel","locus_tag"),all.x = TRUE)%>%filter(locus_tag!="NA")%>%filter(is.na(nonmodel))%>%select(-nonmodel)

In [23]:
nrow(TC)

[1] 146417

In [24]:
LFDR <- TC%>%group_by(Effect)%>%mutate(coeff_log2value=as.numeric(Estimate)/log(2))%>%mutate(Norm_effect=normalize(coeff_log2value))%>%drop_na(Norm_effect)%>%mutate(fdr=(fdrtool(Norm_effect,cutoff.method="locfdr",plot=FALSE,verbose=FALSE)$lfdr))

In [25]:
f_genes_df <- filter_genes%>%filter(Annotation=="Conditionally_Essential"|Annotation== "Essential")%>%select(strain,condition,slevel,locus_tag,Median,controlMedian,mean,controlmean)

In [26]:
TC_results <- merge(filter_genes,LFDR,by=c("locus_tag","strain","condition","slevel"),all.x=TRUE)%>%mutate( Annotation = case_when(
                              fdr.y<0.10 & coeff_log2value >0 ~ "Conditionally_Detrimental",
                              fdr.y<0.10 & coeff_log2value <0 ~ "Conditionally_Beneficial",fdr.y>=0.10~"Conditionally_Neutral",TRUE ~ as.character(Annotation)))%>%select(locus_tag,strain,condition,
                              slevel,Annotation,Median,controlMedian,fdr.y,coeff_log2value,fdr.y)%>%dplyr::rename(fdr=fdr.y)%>%filter(condition!="none")

In [27]:
write.table(TC_results,"../../reports/Master Table/Total_Counts_FDR_v2.TAB",quote=FALSE,sep="\t")

In [28]:
TC_results%>%filter(Annotation!="Conditionally_Neutral")%>%filter(fdr>0.10)

locus_tag,strain,condition,slevel,Annotation,Median,controlMedian,fdr,coeff_log2value
<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
